### Import and Organise the data####

In [1]:
from sklearn.datasets import fetch_mldata
import numpy as np
from sklearn.utils import shuffle
mnist = fetch_mldata('MNIST original', data_home=".")
mnist.data, mnist.target = shuffle(mnist.data, mnist.target)
X = mnist.data
y_float = mnist.target
y_int = y_float.astype(np.int64)
y = np.eye(10)[y_int]

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
s_train = np.ones(X_train.shape[0])
X_train = np.hstack((X_train, s_train.reshape(len(s_train),1)))
s_test = np.ones(X_test.shape[0])
X_test = np.hstack((X_test, s_test.reshape(len(s_test),1)))

## define useful functions 

In [3]:
 #def linear_output_z1(x, w1):
#     z1 = np.dot(x, w1)
#     return z1

 #def logistic_output_z2(x, w2):
#     z2 = sigmoid(np.dot(x, w2))
#     return z2

def sigmoid(a):
    return 1.0 / (1.0 + np.exp(-a))

#def mat_conver_exp(W,X)  

### gradient descent

In [ ]:
# def logistic_output(x, z1, z2, theta1, theta2):
#     zf = np.matmul(z1, theta1) + np.matmul(z2, theta2)
#     return zf

# def gradient_c_theta(z, zf, y):
#     gradient = np.matmul(np.transpose(z), (sigmoid(zf)-y)) / N
#     return gradient

# def gradient_c_w1(x, theta1, zf, y):
#     gradient = np.matmul(np.transpose(x), np.transpose(np.matmul(theta1, np.transpose(np.array((sigmoid(zf) - y)))))) / N
#     return gradient

# def gradient_c_w2(x, w2, theta2, zf, y):
#     elemsig = sigmoid(np.matmul(x, w2)) # [5000, 1]
#     elem1=np.transpose(np.array(sigmoid(zf) - y)) # [10,5000]
#     elem2=np.matmul(theta2, elem1) # [1, 5000]
#     elem3=np.matmul(np.transpose(elemsig), np.transpose(elem2)) # [1,1]
#     elem4 = np.matmul(elemsig, (1 - elem3)) # [5000,1]
#     gradient= np.matmul(np.transpose(x), elem4)/ N # [785,1]

#     return gradient

In [14]:
def gradient(x, y, w1, w2, theta1, theta2):
    z1 = np.matmul(x, w1)
    z2 = np.matmul(x, w2)
    zf = np.matmul(z1, theta1) + np.matmul(z2, theta2)
    sig = (sigmoid(zf)-y) # [N, 10]
    gradient_c_theta1 = np.matmul(np.transpose(z1), sig) / N #[S1, N] [N, 10]
    gradient_c_theta2 = np.matmul(np.transpose(z2), sig) / N #[S2, N] [N, 10]
    gradient_c_z1 = np.matmul(theta1, np.transpose(sig))  / N #[S1, 10] [N, 10]
    gradient_c_z2 = np.matmul(theta2, np.transpose(sig)) / N 
    #gradient_z1_w1= np.tranpose(x).T[:,:,None]*np.transpose(np.multiply(sigmoid(z2), (1-sig(z2)))).T[:,None] #[N, S1, 780]
    gradient_z1_w1 =x
    gradient_c_w1 = np.matmul(np.transpose(x),np.transpose(gradient_c_z1))
    gradient_z2_w2=np.ones((N,S2_logistic,785))
    gradient_c_w2=np.ones((S2_logistic,785))
    for i in range(S2_logistic):
        siggy = np.multiply(sigmoid(z2),(1-sigmoid(z2))) ### [N,S2 ]
        gradient_z2_w2[:,i,:] = x * (siggy[:,i].reshape(N,1))   ###[N,S2,785]
    for i in range(S2_logistic):    
        gradient_c_w2[i,:] =np.matmul(gradient_c_z1[i,:],gradient_z2_w2[:,i,:])
        
    return gradient_c_theta1, gradient_c_theta2, gradient_c_w1, gradient_c_w2

In [15]:
# sig = (sigmoid(zf)-y_train)
# aa = X_train #[N, 785]
# print(aa.shape)
# bb = np.multiply(sigmoid(z2), (1-sigmoid(z2))) #[N,S2]
# print(bb.shape)
# aaa = np.transpose(aa).T[:,:,None] # (52500, 785, 1)
# print(aaa.shape)
# bbb = np.transpose(bb).T[:,None] # (52500, 1, S2)
# print(bbb.shape)
# hogeeeee =aaa*bbb
# print(hogeeeee.shape)

In [16]:
gradient(X_train, y_train, int_w1, int_w2, int_theta1, int_theta2)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


(array([[    9.42747353,    37.61538969,    23.75758091,   182.8911112 ,
           672.41912012,   -57.57062004,    24.50400297,   -47.47640412,
           622.19418636,   556.09774032],
        [  145.49871405,    55.58322598,  -161.30964423,  -651.56966565,
         -1253.19184921,    31.67681185,  -289.94579823,   127.05731074,
         -1525.40297261,  -552.16200917],
        [   79.26653777,  -152.978296  ,  -367.82416679,  -607.93525122,
         -1154.00254711,  -168.31250772,  -556.83566851,    -4.40933921,
         -1212.24557312,  -676.20990069],
        [   -4.28772058,  -103.4589194 ,  -219.20628708,   165.17008301,
           125.40952596,   171.40500432,   130.41244021,    66.30128681,
            75.95094266,   282.6205012 ],
        [ -194.63818211,   215.69733243,   318.00827915,   342.16026585,
           624.07727683,   234.20172028,   609.90887639,     3.94482467,
          1020.91158157,   511.31636686]]),
 array([[ -33.51973413,   15.40040099, -165.10944941,  538

In [ ]:
siggy = np.multiply(sigmoid(z2),(1-sigmoid(z2)))

In [ ]:
siggy.shape

In [ ]:
siggyy * X_train

In [ ]:
cz = np.arange(12.).reshape(4,3)
zw = np.arange(24.).reshape(4,3,2)
ass=np.einsum("ij, ikl", cz, zw)
#hogeee = np.einsum('ji, jik', cz, zw )
#hogger = np.einsum('iik',hogeee)
#cz_t = np.transpose(cz)
#cw = np.tensordot(cz_t, zw, axes=2)#[3,2]

In [ ]:
z2.shape

In [ ]:
KK=np.ones([3])
KK[0]

In [ ]:
for i in range(3):
    KK[i]=i

In [ ]:
KK

In [ ]:
int_theta1.shape

In [6]:
z1 = linear_output_z1(X_train, int_w1)
z2 = logistic_output_z2(X_train, int_w2)
zf = logistic_output(X_train, z1, z2, int_theta1, int_theta2)

NameError: name 'linear_output_z1' is not defined

In [ ]:
z1.shape

In [ ]:
def gradient_descent(x, y, int_w1, int_w2, int_theta1, int_theta2, lowtrain, uptrain, iterations,N):

    place_w1=int_w1
    place_w2=int_w2
    place_theta1=int_theta1
    place_theta2=int_theta2
   # place_theta3=int_theta3

    for i in range(iterations):
        trainrate=np.random.uniform(lowtrain,uptrain,1)/np.cbrt(i+1) 
        z1 = linear_output_z1(x, place_w1)
        z2 = logistic_output_z2(x, place_w2)
        zf = logistic_output(x, z1, z2, place_theta1, place_theta2, )
        p = sigmoid(zf)

        gradient_theta1 = gradient_c_theta(z1, zf, y)
        gradient_theta2 = gradient_c_theta(z2, zf, y)
        gradient_w1 = gradient_c_w1(x, place_theta1, zf, y)
        gradient_w2 = gradient_c_w2(x, place_w2, place_theta2, zf, y)
            
        place_w1 = place_w1 - (trainrate * gradient_w1) - (0.1/N)*place_w1 * trainrate
        place_w2 = place_w2 - (trainrate * gradient_w2) - (0.1/N)*place_w2 * trainrate
        place_theta1 = place_theta1 - trainrate*gradient_theta1*0.1 - (0.1/N)*place_theta1 * trainrate 
        place_theta2 = place_theta2 - trainrate*gradient_theta2 - (0.1/N)*place_theta2 * trainrate
    
    return  p, place_theta1, place_theta2, place_w1, place_w2

In [ ]:
def predict_accuracy(xtest, ytest, theta1, theta2, w1, w2):
    z1 = linear_output_z1(xtest, w1)
    z2 = logistic_output_z2(xtest, w2)
    zf = logistic_output(xtest, z1, z2, theta1, theta2)
    p = sigmoid(zf)
    prediction = np.argmax(p, axis=1)
    answer = np.argmax(ytest, axis=1)
    success_rate = answer == prediction
    success_rate = success_rate*1
    prediction_rate = sum(success_rate) / xtest.shape[0]
    
    return prediction_rate

In [5]:
S1_linear = 5
S2_logistic = 2
S3_Gauss = 3
N=52500 # Batch size

int_w1, int_w2,int_w3 = np.random.uniform(-1,1,(X_train.shape[1],S1_linear)), np.random.uniform(-1,1,(X_train.shape[1], S2_logistic)), np.random.uniform(-1,1,(X_train.shape[1], S3_Gauss))
int_theta1, int_theta2, int_theta3 = np.random.uniform(-1,1,(S1_linear,10)), np.random.uniform(-1,1,(S2_logistic,10)), np.random.uniform(-1,1,(S3_Gauss,10))
#result = gradient_descent(X_train, y_train, int_w1, int_w2, int_theta1, int_theta2, 0.00001, 0.0001, 1,N)

In [ ]:
for i in range(100):
    X_train, y_train = shuffle(X_train, y_train)
    X_train_mini, y_train_mini = X_train[0:100], y_train[0:100]
    N = X_train_mini.shape[0]
    int_w1 = result[3]
    int_w2 = result[4]
    int_theta1 = result[1]
    int_theta2 = result[2]
    #int_theta3# = result[3]

    result = gradient_descent(X_train_mini, y_train_mini, int_w1, int_w2, int_theta1, int_theta2, 0.00001, 0.001, 5,N)
    output = predict_accuracy(X_test, y_test, result[1], result[2], result[3], result[4])
    print(i, output)